In [2]:
#Part2 Implementation
#frequent itemset generation and Association rule generation 

#importing the necessary packages for data handling and generating combinations os sets
import pandas as pd
import numpy as np
from itertools import combinations, chain

#load dataset from the specified path
dt = pd.read_csv("/content/association-rule-test-data.txt",delimiter="\t",header=None)  #please change the file path here
genes = dt.iloc[:, :]
transaction =[]
unique_list = set()
infrequent_itemset = []

#Preprocessing of the dataset to convert to the given format by appending G and gene number
for i in range(0,len(genes)):
  temp = []
  for j in range(0,len(genes[i])+1):
    if j == len(genes):
      genes[j][i] = str(genes[j][i])
      temp.append(genes[j][i])
    # genes[i][j] = str("G"+str(i+1)+"_"+genes[i][j])
    else:
      genes[j][i] = (str("G"+str(j+1)+"_"+genes[j][i]))
      temp.append(genes[j][i])
  transaction.append(temp)

#generating candidate set with single unique elements i.e for caase k = 1
for i in transaction:
  for j in i:
    unique_list.add(j)
# print(len(unique_list))
n=len(transaction)
global_record = {}
final_count_list = []
support_dictionary_rules = {}
final_dictionary = {}

#main function of the program that calls frequent itemset generation function and association rule generation functions.
def mainfunction(transaction,unique_list,minsupport):
  tempcount = 0
  final_dictionary = {}
  global_record = {}
  final_count = 0
  infrequent_itemset = []
  temp_dict = {}
  candidates_list = list(unique_list)
  temp_list = candidates_list
  temp_list = list(set(temp_list))
  candidates_list = temp_list


#Finding the frequent itemsets whose length is 1 i.e k=1 frequent item sets
  n=len(transaction)        #finding length of the transaction table
  new_list = []     
  for item in candidates_list:
    count = 0
    for eachrow in transaction:
      if item in eachrow:
        count += 1
    support = count / n
    if support >= minsupport:
      new_list.append(item)
      tempcount += 1
      global_record[item] = support     #storing it in a dictionary
    else:
      infrequent_itemset.append(item)

  final_count = len(new_list)
  final_dictionary[1] = global_record
  print("Support is set to be: "+str(minsupport * 100)+"%")     #printing the count of number of frequents itemsets of length 1
  print("number of length-1 frequent itemsets:",len(new_list))
  support_dictionary_rules = global_record
  final_count_list.append(new_list)

#Function call for generating subsequent frequent itemsets of k- lengths
  final_dictionary, support_dictionary_rules = frequentset_k(new_list,unique_list,final_count,final_dictionary,support_dictionary_rules)

#function call for generating the association rule given the total frequent itemsset
  association_rules = rules_generation(minconfidence,final_dictionary,transaction,support_dictionary_rules)
  
  # print("\n")
  # print("number of association rules generated", len(association_rules))

  return association_rules

#function definition for frequent itemset generation for k lengths
def frequentset_k(new_list,unique_list,final_count,final_dictionary,support_dictionary_rules):
  final_count_return = final_count
  new_candidateset = []
  candidates_list = []
  ans = []
  temp_list = []
  for i in range(2,len(new_list)+1):
    frequent_itemset = []
    temp_suport_dict = {}
    for j in range(0,len(new_list)):

      for k in range(j+1,len(new_list)):
        value = ""
        newset = list(set(new_list[j].split(",") + new_list[k].split(",")))
        final = list(set(newset))
        item_len = len(final)

        if(i == item_len):
          word = ""
          word = ",".join(sorted(final))
          ans.append(word)
          # mas_dic[word] = 0
          temp_list.append(word)

    temp_list = list(set(temp_list))
    candidates_list = temp_list
    temp_list = []    
    

    for item in candidates_list:
      c_list = item.split(",")
      # print(item)
      count = 0
      for eachrow in transaction:
        if(set(c_list).issubset(set(eachrow))):
          count += 1
      support = count / n
      if support >= minsupport:
        frequent_itemset.append(item)
        global_record[item] = support 
        support_dictionary_rules[item] = support
        temp_suport_dict[item] = support       
      else:
        infrequent_itemset.append(item)
    final_dictionary[i] = temp_suport_dict

    final_count_list.append(frequent_itemset)

    if(len(frequent_itemset)==0):
      break
    new_list = frequent_itemset
    final_count_return += len(frequent_itemset)
    print("number of length-"+str(i)+"frequent itemsets: ",len(frequent_itemset))
  print("number of all lengths frequent itemsets: ",final_count_return)

  return final_dictionary, support_dictionary_rules


In [3]:
def rules_generation(minconfidence,final_dictionary,transaction,support_dictionary_rules):
  import itertools
  confident_set = 0
  association_rules = set()
  for itemlength in final_dictionary.keys():
    if itemlength > 1:
      for itemset in final_dictionary[itemlength]:
        lhs_value = 100 * support_dictionary_rules[itemset]
        lhs_each_value = itemset.split(",")
        if lhs_value != 0:
          for k in range(len(lhs_each_value)-1, 0,-1):
            for each_k in list(itertools.combinations(lhs_each_value,k)):
              lhs = list(set(list(each_k))^set(lhs_each_value))
              rule = ",".join(lhs)+"->"+ ",".join(list(each_k))
              temp = list(each_k)
              if rule not in association_rules:
                rhs_count = 0
                for eachrow in transaction:
                  if(set(temp).issubset(eachrow)):
                    rhs_count += 1
                if(((lhs_value/rhs_count) * 100 ) / 100 >= minconfidence):
                  confident_set += 1
                  association_rules.add(rule) 
              
  return association_rules


  

In [4]:
minsupport = 50
minconfidence = 70
minsupport = minsupport/100
minconfidence = minconfidence/100
association_rules = mainfunction(transaction,unique_list,minsupport)

#association_rules = rules_generation(minconfidence,data,transaction,support_dictionary_rules)

Support is set to be: 50.0%
number of length-1 frequent itemsets: 109
number of length-2frequent itemsets:  63
number of length-3frequent itemsets:  2
number of all lengths frequent itemsets:  174


In [6]:
#template1 implementation


def call_template1():
  query = input("enter the inputs in specified format semi-colon seperated: (for ex); RULE;ANY;G59_Up,G72_Up ")
  output = template1(association_rules,query)
  print("Count of the query returned: ", len(output))
  print("Rules returned: ")
  for rule in output:
    print(rule)


#this function has to return count of query in template1
def template1(ruleList, query):
  output_rules = set()
  query_input = query.split(";")  
  type1 = query_input[0]
  type2 = query_input[1]
  query = query_input[2].split(",")
  final_rule_list = []
  confidence_ct = 0
  if (type1.upper() == 'RULE' and type2.upper()  == 'ANY' ):     
      for rule in ruleList:
        temp = []
        final_rule_list = []
        count = 0
        rule_list = rule.split("->")
        final_rule_list.append(rule_list[0].split(","))
        final_rule_list.append(rule_list[1].split(","))
        for i in final_rule_list:
          for j in i:
            temp.append(j)
        for q in query:
          if q in temp:
            count += 1
            confidence_ct += 1
            output_rules.add(rule)
            break;

  elif (type1.upper() == 'RULE' and type2.upper() == 'NONE' ): 
    #print("Printing rule and none")
    confidence_ct = 0
    for rule in ruleList:
      temp = []
      final_rule_list = []
      count = 0
      rule_list = rule.split("->")
      final_rule_list.append(rule_list[0].split(","))
      final_rule_list.append(rule_list[1].split(","))
      
      for i in final_rule_list:
        for j in i:
          temp.append(j)
      for q in query:
        if q not in temp:
          count += 1
      if(count == len(query)):
        confidence_ct += 1
        output_rules.add(rule)
    
    #print(confidence_ct)

  elif (type1.upper() == 'RULE' and (type2 != 'ANY' or type2 != 'NONE')):      
      confidence_ct = 0
      for rule in ruleList:
        count = 0
        temp = []
        final_rule_list = []
        count = 0
        rule_list = rule.split("->")
        final_rule_list.append(rule_list[0].split(","))
        final_rule_list.append(rule_list[1].split(","))
        
        for i in final_rule_list:
          for j in i:
            temp.append(j)
        for q in query:
          if q in temp:
            count += 1
        if type2 == '1':
          if count == 1 and len(query) > 1:
            confidence_ct = confidence_ct+1
            output_rules.add(rule)
          elif count == 1 and len(query) == 1:
            confidence_ct = confidence_ct+1
            output_rules.add(rule)

        elif type2 != '1':
          if count ==  int(type2) and len(query) == int(type2):
            confidence_ct = confidence_ct+1
            output_rules.add(rule)

       

      #print(confidence_ct)

  elif (type1.upper() == 'HEAD' and type2.upper() == 'ANY'):    
      for rule in ruleList:
        head = rule.split('->')[0]
        head1 = head.split(',')
        for q in query:
          if q in head1:
            confidence_ct = confidence_ct+1
            output_rules.add(rule)
          break
                  

  elif (type1.upper() == 'HEAD' and type2.upper() == 'NONE'):     
    for rule in ruleList:
      count = 0
      head = rule.split('->')[0]
      head1 = head.split(',')
      for q in query:
        if q not in head1:
          count += 1
      if count == len(query):
        confidence_ct = confidence_ct+1
        output_rules.add(rule)

  elif (type1.upper() == 'HEAD' and (type2 != 'ANY' or type2 != 'NONE')):     
        confidence_ct = 0
        for rule in ruleList:
          count = 0
          head = rule.split('->')[0]
          head1 = head.split(',')
          for q in query:
            if q in head1:
              count += 1
          if type2 == '1':
            if count == 1 and len(query) > 1:
              confidence_ct = confidence_ct+1
              output_rules.add(rule)
            elif count == 1 and len(query) == 1:
              confidence_ct = confidence_ct+1
              output_rules.add(rule)

          elif type2 != '1':
            if count ==  int(type2) and len(query) == int(type2):
              confidence_ct = confidence_ct+1
              output_rules.add(rule)            
        
  elif (type1.upper() == 'BODY' and type2.upper() == 'ANY'):     
      for rule in ruleList:
        body = rule.split('->')[1]
        body1 = body.split(',')
        for q in query:
          if (q in body1):
            confidence_ct = confidence_ct+1
            output_rules.add(rule)
          break

  elif (type1.upper() == 'BODY' and type2.upper() == 'NONE'):      
    for rule in ruleList:
      count = 0
      body = rule.split('->')[1]
      body1 = body.split(',')
      for q in query:
        if q not in body1:
          count += 1
      if count == len(query):
        confidence_ct = confidence_ct+1
        output_rules.add(rule)


  elif (type1.upper() == 'BODY' and (type2 != 'ANY' or type2 != 'NONE')):     
        confidence_ct = 0
        for rule in ruleList:
          count = 0
          body = rule.split('->')[1]
          body1 = body.split(',')
          for q in query:
            if q in body1:
              count += 1
          if type2 == '1':
            if count == 1 and len(query) > 1:
              confidence_ct = confidence_ct+1
              output_rules.add(rule)
            elif count == 1 and len(query) == 1:
              confidence_ct = confidence_ct+1
              output_rules.add(rule)

          elif type2 != '1':
            if count ==  int(type2) and len(query) == int(type2):
              confidence_ct = confidence_ct+1
              output_rules.add(rule)                 

  return list(output_rules)
         
call_template1()

enter the inputs in specified format semi-colon seperated: (for ex); RULE;ANY;G59_Up,G72_Up BODY;1;G59_Up,G10_Down
Count of the query returned:  17
Rules returned: 
G72_Up->G59_Up,G96_Down
G72_Up->G59_Up
G59_Up->G10_Down
G1_Up->G10_Down
G82_Down->G59_Up,G72_Up
G70_Down->G10_Down
G82_Down->G59_Up
G96_Down->G59_Up,G72_Up
G28_Down->G10_Down
G72_Up->G59_Up,G82_Down
G88_Down->G10_Down
G38_Down->G10_Down
G88_Down->G59_Up
G47_Up->G10_Down
G96_Down->G59_Up
G13_Down->G59_Up
G94_Up->G10_Down


In [7]:
#template2 implementation
#this function has to return rules of size greater than or equal to the given number
def call_template2():
  query = input("enter the input in comma separated form (for ex) RULE;2 ")
  output = template2(association_rules,query)
  print("Count of the query returned: ", len(output))
  print("Rules returned: ")
  for rule in output:
    print(rule)



def template2(association_rules,query):
  query_input = query.split(";")
  # query_input[0] = query_input[0].toUpper() 
  size = int(query_input[1])
  count = 0
  output_rules = set()
  if query_input[0] == "RULE":
    for eachrule in association_rules:
      rule_list = eachrule.split("->")
      head_items = rule_list[0].split(",")  
      body_items = rule_list[1].split(",")  
      length = len(head_items) + len(body_items)
      if length >= size:
        count +=1
        output_rules.add(eachrule)

  elif query_input[0] == "HEAD":
    for eachrule in association_rules:
      rule_list = eachrule.split("->")
      head_items = rule_list[0].split(",") 
      if len(head_items) >= size:
        count += 1
        output_rules.add(eachrule) 
 
  elif query_input[0] == "BODY":
    for eachrule in association_rules:
      rule_list = eachrule.split("->")
      body_items = rule_list[1].split(",") 
      if len(body_items) >= size:
        count += 1
        output_rules.add(eachrule) 

  # print("query result count:",count)
  # print(len(output_rules))
  return list(output_rules)

call_template2()



enter the input in comma separated form (for ex) RULE;2 HEAD;2
Count of the query returned:  3
Rules returned: 
G59_Up,G72_Up->G82_Down
G59_Up,G82_Down->G72_Up
G59_Up,G72_Up->G96_Down


In [8]:
def  template3(association_rules):
  query_input = input("Enter the input in the given format of query (for example:) 1,or,1|HEAD;ANY;G10_Down,G9_Up|BODY;1;G59_Up ")
  query_input_split = query_input.split("|")
  query_type = query_input_split[0]
  query_type_list = query_type.split(",")
  output1 = set()
  output2 = set()

  if int(query_type_list[0]) == 1 and query_type_list[1] == "or" and int(query_type_list[2]) == 1: #works fine
    output1 = template1(association_rules,query_input_split[1])  
    output2 = template1(association_rules,query_input_split[2])
    final_result = (set(output1) | set(output2))

  elif int(query_type_list[0]) == 1 and query_type_list[1] == "or" and int(query_type_list[2]) == 2:  #works fine
    output1 = template1(association_rules,query_input_split[1])  
    output2 = template2(association_rules,query_input_split[2])
    final_result = (set(output1) | set(output2))
  
  elif int(query_type_list[0]) == 1 and query_type_list[1] == "and" and int(query_type_list[2]) == 1:  #works fine
    output1 = template1(association_rules,query_input_split[1])  
    output2 = template1(association_rules,query_input_split[2])
    final_result = (set(output1) & set(output2))

  elif int(query_type_list[0]) == 1 and query_type_list[1] == "and" and int(query_type_list[2]) == 2:  #works fine
    output1 = template1(association_rules,query_input_split[1])  
    output2 = template2(association_rules,query_input_split[2])
    final_result = (set(output1) & set(output2))
  
  elif int(query_type_list[0]) == 2 and query_type_list[1] == "or" and int(query_type_list[2]) == 2:  #works fine
    output1 = template2(association_rules,query_input_split[1])  
    output2 = template2(association_rules,query_input_split[2])
    final_result = (set(output1) | set(output2))

  elif int(query_type_list[0]) == 2 and query_type_list[1] == "and" and int(query_type_list[2]) == 2:  #works fine
    output1 = template2(association_rules,query_input_split[1])  
    output2 = template2(association_rules,query_input_split[2])
    final_result = (set(output1) & set(output2))
 
  elif int(query_type_list[0]) == 2 and query_type_list[1] == "or" and int(query_type_list[2]) == 1:
    output1 = template2(association_rules,query_input_split[1])  
    output2 = template1(association_rules,query_input_split[2])
    final_result = (set(output1) | set(output2))

  elif int(query_type_list[0]) == 2 and query_type_list[1] == "and" and int(query_type_list[2]) == 1:
    output1 = template2(association_rules,query_input_split[1])  
    output2 = template1(association_rules,query_input_split[2])
    final_result = (set(output1) & set(output2))



  print("Count of the query returned: ",len(final_result))
  return final_result

    
final_val=template3(association_rules)
print("Rules returned: ")
for rule in final_val:
  print(rule)


Enter the input in the given format of query (for example:) 1,or,1|HEAD;ANY;G10_Down,G9_Up|BODY;1;G59_Up 1,and,2|HEAD;ANY;G10_Down|BODY;2
Count of the query returned:  0
Rules returned: 
